In [1]:
import os
os.chdir("../../")

import pandas as pd
from scripts.utils import *
from scripts.data import *

In [2]:
pd.set_option('display.max_colwidth', 1000)

In [3]:
dataset_names = [d.replace(".json", "") for d in os.listdir("data/raw/reasoning") if d.endswith(".json")]
dataset_names

['object_tracking',
 'commonsensqa',
 'aqua',
 'svamp',
 'multiarith',
 'gsm8k',
 'coin_flip',
 'singleeq',
 'strategyqa',
 'addsub',
 'last_letters',
 'bigbench_date']

In [4]:
import random


def load_dataset(dataset_name):
    return read_json("data/raw/reasoning/{}.json".format(dataset_name))


def make_mixed_multi_questions_prompt_sources(datasets, n_q_each=1, n_instances=300, shuffle_q_order=False):

    if n_q_each == 1:
        shuffle_q_order = False

    sources = set()
    for _ in range(n_instances):
       
        while True:
            source = tuple()
            for _ in range(n_q_each):
                random.shuffle(datasets)

                for dataset in datasets:
                    name = dataset["dataset"]
                    ix = random.randint(0, dataset["# samples"]-1)
                    source += ((name, ix), )
            
            if shuffle_q_order:
                source = list(source)
                random.shuffle(source)
                source = tuple(source)

            if source not in sources:
                sources.add(source)
                break

    return tuple(sources)


def make_mixed_questions_from_prompt_sources(sources, method="zero-shot", name_2_dataset_map=None):
    out = []
    qsize = len(sources[0])

    if name_2_dataset_map is None:
        datasets = [load_dataset(name) for name in set([n for n,_ in sources[0]])]
        name_2_dataset_map = {d["dataset"]: d for d in datasets}

    for i, source in enumerate(sources):
        questions, answers = [], []

        for name, ix in source:
            dataset = name_2_dataset_map[name]
            questions.append(dataset["questions"][ix])
            answers.append(dataset["answers"][ix])
        prompt = make_to_multi_question_prompt(questions, method=method)
        out.append([i+1, prompt, answers, source, qsize])
    
    return out


def make_mixed_questions(datasets, n_q_each=1, n_instances=300, method="zero-shot", shuffle_q_order=False):
    cols = ["promptIdx", "prompt", "answer", "source", "qsize"]
    sources = make_mixed_multi_questions_prompt_sources(datasets, n_q_each, n_instances, shuffle_q_order)
    name_2_dataset_map = {d["dataset"]: d for d in datasets}
    out = make_mixed_questions_from_prompt_sources(sources, method, name_2_dataset_map)
    return pd.DataFrame(out, columns=cols)


def generate_dataset_combinations(num_dataset, num_versions=5):
    dataset_names = [d.replace(".json", "") for d in os.listdir("data/raw/reasoning") if d.endswith(".json")]

    assert len(dataset_names) >= 2, "There should be at least 2 datasets"
    assert 2 <= num_dataset <= len(dataset_names), "num_dataset should be between 2 and {}".format(len(dataset_names)) 

    # this five math datasets are all free-response questions
    # although aqua is also a math dataset, it is multiple choice
    math_names = {"gsm8k", "addsub", "multiarith", "svamp", "singleeq"}

    versions = []
    seen_for_the_first_two = set()

    for _ in range(num_versions):
        while True:
            sample_names = random.sample(dataset_names, num_dataset)
            # this will ensure that we don't have two math datasets in the first two of the same version
            if sample_names[0] in math_names and sample_names[1] in math_names:
                pass
            
            # this diversify the first two datasets
            elif sample_names[0] in seen_for_the_first_two or sample_names[1] in seen_for_the_first_two:
                pass
            else:
                versions.append(sample_names)
                seen_for_the_first_two.update(set(sample_names[:2]))
                break
    
    return versions

In [7]:
versions = generate_dataset_combinations(10, 6)

In [8]:
for v in versions:
    print(v)

['singleeq', 'strategyqa', 'addsub', 'multiarith', 'object_tracking', 'aqua', 'last_letters', 'gsm8k', 'commonsensqa', 'coin_flip']
['bigbench_date', 'aqua', 'singleeq', 'gsm8k', 'svamp', 'object_tracking', 'strategyqa', 'coin_flip', 'commonsensqa', 'addsub']
['svamp', 'last_letters', 'commonsensqa', 'aqua', 'strategyqa', 'bigbench_date', 'multiarith', 'object_tracking', 'gsm8k', 'coin_flip']
['commonsensqa', 'addsub', 'strategyqa', 'multiarith', 'last_letters', 'object_tracking', 'aqua', 'svamp', 'coin_flip', 'singleeq']
['coin_flip', 'gsm8k', 'singleeq', 'multiarith', 'addsub', 'last_letters', 'commonsensqa', 'object_tracking', 'aqua', 'strategyqa']
['multiarith', 'object_tracking', 'addsub', 'gsm8k', 'aqua', 'last_letters', 'commonsensqa', 'svamp', 'coin_flip', 'singleeq']


In [9]:
database_10 = {"# datasets": 10, "# of versions": 6}

for i, v in enumerate(versions):
    database_10[f"v#{i+1}"] = dict()
    database_10[f"v#{i+1}"]["datasets"] = v
    database_10[f"v#{i+1}"]["# qsize variants"] = 1
    database_10[f"v#{i+1}"]["qsize=10"] = make_mixed_multi_questions_prompt_sources([load_dataset(d) for d in v], n_q_each=1, n_instances=300, shuffle_q_order=False)

data/raw/reasoning/singleeq.json has been loaded!
data/raw/reasoning/strategyqa.json has been loaded!
data/raw/reasoning/addsub.json has been loaded!
data/raw/reasoning/multiarith.json has been loaded!
data/raw/reasoning/object_tracking.json has been loaded!
data/raw/reasoning/aqua.json has been loaded!
data/raw/reasoning/last_letters.json has been loaded!
data/raw/reasoning/gsm8k.json has been loaded!
data/raw/reasoning/commonsensqa.json has been loaded!
data/raw/reasoning/coin_flip.json has been loaded!
data/raw/reasoning/bigbench_date.json has been loaded!
data/raw/reasoning/aqua.json has been loaded!
data/raw/reasoning/singleeq.json has been loaded!
data/raw/reasoning/gsm8k.json has been loaded!
data/raw/reasoning/svamp.json has been loaded!
data/raw/reasoning/object_tracking.json has been loaded!
data/raw/reasoning/strategyqa.json has been loaded!
data/raw/reasoning/coin_flip.json has been loaded!
data/raw/reasoning/commonsensqa.json has been loaded!
data/raw/reasoning/addsub.json

In [10]:
os.makedirs("data/databases/reasoning/mixed/", exist_ok=True)

save_obj_as_pickle(database_10, "data/databases/reasoning/mixed/database_10.pkl")

Saved object to data/databases/reasoning/mixed/database_10.pkl


In [12]:
def trim(sources, dataset_names):
    out = tuple()
    for source in sources:
        new_source = []
        for name, ix in source:
            if name in dataset_names:
                new_source.append((name, ix))
        out += (tuple(new_source), )

    return out


for l in [8, 6, 4, 2]:
    database_l = {"# datasets": l, "# of versions": 5}
    for i in range(database_10["# of versions"]):
        database_l[f"v#{i+1}"] = dict()
        dataset_names = database_10[f"v#{i+1}"]["datasets"][:l]
        database_l[f"v#{i+1}"]["datasets"] = dataset_names
        database_l[f"v#{i+1}"]["# qsize variants"] = 1
        sources = database_10[f"v#{i+1}"][f"qsize=10"]
        database_l[f"v#{i+1}"][f"qsize={l}"] = trim(sources, dataset_names)
    
    save_obj_as_pickle(database_l, f"data/databases/reasoning/mixed/database_{l}.pkl")

Saved object to data/databases/reasoning/mixed/database_8.pkl
Saved object to data/databases/reasoning/mixed/database_6.pkl
Saved object to data/databases/reasoning/mixed/database_4.pkl
Saved object to data/databases/reasoning/mixed/database_2.pkl


### Make mixed-questions prompts

In [21]:
os.makedirs("results/reasoning/mixed/", exist_ok=True)
cols = ["promptIdx", "prompt", "answer", "source", "qsize"]

for l in [10, 8, 6, 4, 2]:
    os.makedirs(f"results/reasoning/mixed/#datasets={l}/", exist_ok=True)
    database = read_obj_from_pickle(f"data/databases/reasoning/mixed/database_{l}.pkl")

    for i in range(len(versions)):
        datasets = database[f"v#{i+1}"]["datasets"]
        sources = database[f"v#{i+1}"][f"qsize={l}"]
        os.makedirs(f"results/reasoning/mixed/#datasets={l}/v#{i+1}/", exist_ok=True)

        for method in ["zero-shot", "zero-shot-cot"]:
            out = make_mixed_questions_from_prompt_sources(sources, method)
            df = pd.DataFrame(out, columns=cols)

            df.to_json(f"results/reasoning/mixed/#datasets={l}/v#{i+1}/{method}.json", orient="records", indent=4)
            print(f"results/reasoning/mixed/#datasets={l}/v#{i+1}/{method}.json" + " created!")

Read object from data/databases/reasoning/mixed/database_10.pkl
data/raw/reasoning/singleeq.json has been loaded!
data/raw/reasoning/strategyqa.json has been loaded!
data/raw/reasoning/object_tracking.json has been loaded!
data/raw/reasoning/last_letters.json has been loaded!
data/raw/reasoning/commonsensqa.json has been loaded!
data/raw/reasoning/aqua.json has been loaded!
data/raw/reasoning/coin_flip.json has been loaded!
data/raw/reasoning/addsub.json has been loaded!
data/raw/reasoning/multiarith.json has been loaded!
data/raw/reasoning/gsm8k.json has been loaded!
results/reasoning/mixed/#datasets=10/v#1/zero-shot.json created!
data/raw/reasoning/singleeq.json has been loaded!
data/raw/reasoning/strategyqa.json has been loaded!
data/raw/reasoning/object_tracking.json has been loaded!
data/raw/reasoning/last_letters.json has been loaded!
data/raw/reasoning/commonsensqa.json has been loaded!
data/raw/reasoning/aqua.json has been loaded!
data/raw/reasoning/coin_flip.json has been load

data/raw/reasoning/multiarith.json has been loaded!
results/reasoning/mixed/#datasets=6/v#4/zero-shot.json created!
data/raw/reasoning/strategyqa.json has been loaded!
data/raw/reasoning/object_tracking.json has been loaded!
data/raw/reasoning/last_letters.json has been loaded!
data/raw/reasoning/commonsensqa.json has been loaded!
data/raw/reasoning/addsub.json has been loaded!
data/raw/reasoning/multiarith.json has been loaded!
results/reasoning/mixed/#datasets=6/v#4/zero-shot-cot.json created!
data/raw/reasoning/singleeq.json has been loaded!
data/raw/reasoning/last_letters.json has been loaded!
data/raw/reasoning/coin_flip.json has been loaded!
data/raw/reasoning/addsub.json has been loaded!
data/raw/reasoning/multiarith.json has been loaded!
data/raw/reasoning/gsm8k.json has been loaded!
results/reasoning/mixed/#datasets=6/v#5/zero-shot.json created!
data/raw/reasoning/singleeq.json has been loaded!
data/raw/reasoning/last_letters.json has been loaded!
data/raw/reasoning/coin_flip.

#### Sanity check

In [43]:
# between methods 

tmp = "results/reasoning/mixed/#datasets={}/v#{}/{}.json"

for v in [1, 2, 3, 4, 5, 6]:
    for l in [2, 4, 6, 8, 10]:
        zero_shot = tmp.format(l, v, "zero-shot")
        zero_shot_cot = tmp.format(l, v, "zero-shot-cot")
        zero_shot_df = pd.read_json(zero_shot)
        zero_shot_cot_df = pd.read_json(zero_shot_cot)
        assert len(zero_shot_df) == 300
        assert zero_shot_df[["answer", "source", "qsize"]].equals(zero_shot_cot_df[["answer", "source", "qsize"]])

In [39]:
# among lengths: for each promptIdx, sources in the shorter length is a subset of the longer length

lengths = [2, 4, 6, 8, 10]
for v in [1, 2, 3, 4, 5, 6]:
    for i in range(len(lengths)):
        l1 = lengths[i]

        for j in range(i+1, len(lengths)):
            l2 = lengths[j]

            df1 = pd.read_json(tmp.format(l1, v, "zero-shot"))
            df2 = pd.read_json(tmp.format(l2, v, "zero-shot"))

            for idx in df1["promptIdx"]:
                sources1 = df1[df1["promptIdx"] == idx]["source"].values[0]
                sources2 = df2[df2["promptIdx"] == idx]["source"].values[0]

                sources1 = [tuple(s) for s in sources1]
                sources2 = [tuple(s) for s in sources2]

                assert set(sources1) <= set(sources2)

In [40]:
## manual

df1.sample(5)

,promptIdx,prompt,answer,source,qsize
275,276,"Questions\n\nQ1: Alice, Bob, and Claire are on the same team in a soccer match. At the start of the match, they are each assigned to a position: Alice is playing right winger, Bob is playing left midfielder, and Claire is playing center midfielder. \n\nAs the game progresses, pairs of players occasionally swap positions. First, Claire and Alice trade positions. Then, Bob and Alice trade positions. Finally, Claire and Bob trade positions. At the end of the match, Alice is playing \nWhich choice is true ? Answer Choices: (A) right winger. (B) left midfielder. (C) center midfielder.\n\nQ2: Alani's family decided that the children should write stories of any kind. They were then required to read all of the stories they'd written to the family at the end of the weekend. Alani wrote 20 stories in the first week, her brother Braylen wrote 40 stories, and her sister Margot wrote 60 stories. If they each doubled the number of stories they'd written in the first week in the second week, calc...","[B, 360, 26, diln, 5, 42.33, D, D]","[[object_tracking, 596], [gsm8k, 620], [svamp, 261], [last_letters, 218], [multiarith, 396], [addsub, 356], [commonsensqa, 212], [aqua, 97]]",8
98,99,"Questions\n\nQ1: There school's baseball team had 12 new players and 44 returning players. If the coach put them into groups with 8 players in each group, how many groups would there be?\n\nQ2: If 75 percent of the employees of a certain company take a winter vacation, 40 percent take a winter and a summer vacation, and 20 percent take neither a winter nor a summer vacation, what Q percent of the employees take a summer vacation but not a winter vacation?\nAnswer Choices: (A) 5% (B) 15% (C) 25% (D) 35% (E) 45%\n\nQ3: Alice, Bob, and Claire are holding a white elephant gift exchange. At the start of the event, they are each holding a present of a different color: Alice has a blue present, Bob has a orange ball, and Claire has a brown present. \n\nAs the event progresses, pairs of people swap gifts. First, Bob and Alice swap their gifts. Then, Claire and Alice swap their gifts. Finally, Claire and Bob swap their gifts. At the end of the event, Claire has the \nWhich choice is true ? ...","[7, A, A, ghir, 0.3333333333333333, 15, B, 38]","[[multiarith, 340], [aqua, 180], [object_tracking, 511], [last_letters, 450], [addsub, 192], [svamp, 505], [commonsensqa, 792], [gsm8k, 194]]",8
206,207,"Questions\n\nQ1: When a human is earning money, where are they often found?\nAnswer Choices: (A) deep thought (B) park (C) friend's house (D) place of work (E) school\n\nQ2: Marco and his dad went strawberry picking. Together their strawberries weighed 20 pounds. His dad's strawberries weighed 17 pounds. How much did marco's strawberries weigh?\n\nQ3: My wall contains 8 red colour ties, 13 violet colour ties,10 blue colour ties, 5 pink colour ties, 4 green colour ties. If electricity is gone and I want at least two ties of same colour then how many ties I should take out from my rack?\nAnswer Choices: (A) 2 (B) 3 (C) 4 (D) 5 (E) 6\n\nQ4: Ralph has a record store where people can trade their own records for new ones. People can trade 2 old records for 1 new one. 5 people come in with old records and leave with 7 new records between them. How many old records did the 5 people bring in?\n\nQ5: Alice, Bob, and Claire are friends and avid readers who occasionally trade books. At the sta...","[D, 3, E, 14, A, ayea, 12, 22]","[[commonsensqa, 1140], [svamp, 864], [aqua, 233], [gsm8k, 374], [object_tracking, 510], [last_letters, 175], [multiarith, 347], [addsub, 171]]",8
225,226,"Questions\n\nQ1: At my favorite fruit stand, an orange costs 18 dollars, a pineapple costs 27 dollars, and a grape costs 15 dollars. Using the same logic, can you tell how much a mango costs?\nAnswer Choices: (A) 22 dollars (B) 15 dollars (C) 20 dollars (D) 18 dollars (E) 10 dollars\n\nQ2: There are 17 students trying out for the school's trivia teams. If 5 of them 

In [42]:
ix = 276

print(df1[df1["promptIdx"] == ix]["prompt"].values[0])
print(df1[df1["promptIdx"] == ix]["answer"].values[0])
print(df1[df1["promptIdx"] == ix]["source"].values[0])

Questions

Q1: Alice, Bob, and Claire are on the same team in a soccer match. At the start of the match, they are each assigned to a position: Alice is playing right winger, Bob is playing left midfielder, and Claire is playing center midfielder. 

As the game progresses, pairs of players occasionally swap positions. First, Claire and Alice trade positions. Then, Bob and Alice trade positions. Finally, Claire and Bob trade positions. At the end of the match, Alice is playing 
Which choice is true ? Answer Choices: (A) right winger. (B) left midfielder. (C) center midfielder.

Q2: Alani's family decided that the children should write stories of any kind. They were then required to read all of the stories they'd written to the family at the end of the weekend. Alani wrote 20 stories in the first week, her brother Braylen wrote 40 stories, and her sister Margot wrote 60 stories. If they each doubled the number of stories they'd written in the first week in the second week, calculate the t

#### Evaluation beta

In [151]:
import re
from scripts.evaluate import answer_cleansing


def get_preds(pred, qsize=2):
    preds = []
    if pred[0] == ":" and len(pred) > 1:
        pred = pred[1:]
    
    pred = pred.strip()
    if len(pred.split("\n\n")) == qsize:
        preds = pred.split("\n\n")
    
    elif len(pred.split("\n")) == qsize:
        preds = pred.split("\n")

    elif len(re.findall(r"[AQ]\d*:.*", pred)) == qsize:
        preds = re.findall(r"[AQ]\d*:.+", pred)
    
    if len(preds) != qsize:
        return False
    
    return [re.sub(r"[AQ]\d*:", "", p).strip() for p in preds]

In [154]:
# answer_cleansing(p, method, dataset, directAnswerTrigger)

for ix in o.index:
    preds = get_preds(o.loc[ix, f"{model}-completion"], qsize=qsize)
    if preds:
        for i, pred in enumerate(preds):
            preds[i] = answer_cleansing(pred, "zero-shot", o.loc[ix, "source"][i][0], None)
    
        o.at[ix, f"{model}-preds"] = "\n".join(preds)

In [155]:
o

,promptIdx,prompt,answer,source,qsize,gpt-3.5-turbo-0125-completion,gpt-3.5-turbo-0125-preds
0,1,"Questions\n\nQ1: The recent floods in Mamou’s country have left many families without food or shelter. To help, Mamou has volunteered to distribute 1,360 meals to the affected families. She gave out 64 meals on Friday, 30 meals on Saturday, and 48 meals on Sunday. How many meals does she have left to distribute?\n\nQ2: A coin is heads up. Kennedy flips the coin. Ginny does not flip the coin. Iliana flips the coin. Sky does not flip the coin. Is the coin still heads up? Note that ""flip"" here means ""reverse"".\n\nQ3: What is a prisoner sentenced to do?\nAnswer Choices: (A) scape jail (B) dream of freedom (C) become a hairdresser (D) attempt to escape (E) do time\n\nQ4: Take the last letters of each words in ""Max Mikey Cynthia Holly"" and concatenate them.\n\nQ5: Professors borrowed Rs. 5000 from the university at simple interest. After 3 years, the university got Rs. 300 on interest. What was the rate of interest per annum?\nAnswer Choices: (A) 2% (B) 8% (C) 5% (D) 10% (E) None of thes...","[1218, yes, E, xyay, A, 34]","((gsm8k, 1146), (coin_flip, 487), (commonsensqa, 754), (last_letters, 115), (aqua, 241), (singleeq, 271))",6,"A1: Mamou has 1,218 meals left to distribute.\nA2: Yes, the coin is still heads up.\nA3: (E) do time\nA4: ""yayy""\nA5: (B) 8%\nA6: Dan gave Jessica 34 seashells.",1218\nyes\nE\nyayy\nB\n34
1,2,"Questions\n\nQ1: What would a person do if they do not have any friends?\nAnswer Choices: (A) talk to people (B) try again (C) fall asleep (D) stand alone (E) thank god\n\nQ2: A coin is heads up. Dusty does not flip the coin. Yanet flips the coin. Hortencia flips the coin. Lili flips the coin. Is the coin still heads up? Note that ""flip"" here means ""reverse"".\n\nQ3: A juggler can juggle 16 balls. Half of the balls are golf balls, and half of the golf balls are blue. How many blue golf balls are there?\n\nQ4: Joan has 9 blue balloons, Sally has 5 blue balloons, and Jessica has 2 blue balloons. How many blue balloons do they have in total ?\n\nQ5: Take the last letters of each words in ""Fermin Berenice Marianne Eleanor"" and concatenate them.\n\nQ6: Two cars start at the same time from opposite ends of a highway that is 50 miles long. One car is riding at 12 mph and the second car is riding at 13 mph. How long after they begin will they meet?\nAnswer Choices: (A) 1 (B) 1.25 (C) 1...","[D, no, 4, 16, neer, E]","((commonsensqa, 312), (coin_flip, 259), (gsm8k, 35), (singleeq, 294), (last_letters, 376), (aqua, 175))",6,"A1: (A) talk to people\nA2: Yes, the coin is still heads up\nA3: 4 blue golf balls\nA4: 16 blue balloons in total\nA5: Nene\nA6: (C) 1.50 hours",A\nyes\n4\n16\nNene\nC
2,3,"Questions\n\nQ1: There are 1110 students at a school. If each classroom holds 30 students, how many classrooms are needed at the school?\n\nQ2: What does a self assured person often do?\nAnswer Choices: (A) acknowledgment (B) focused (C) know what time (D) feel important (E) trust himself\n\nQ3: A coin is heads up. Garrett does not flip the coin. Eva does not flip the coin. Joaquin flips the coin. Monique does not flip the coin. Is the coin still heads up? Note that ""flip"" here means ""reverse"".\n\nQ4: Take the last letters of each words in ""Cassie Clifton Erik Everardo"" and concatenate them.\n\nQ5: Two sisters, Elizabeth and Margareth, bought beads. Elizabeth bought 1 pack of red and 2 packs of clear beads, while Margareth bought 3 packs of blue and 4 packs of red beads. How many more beads do one sister have than the other, if each pack of beads contains 20 pieces of beads?\n\nQ6: The original price of an item is discounted 22%. A customer buys the item at this discounted price us...","[37, D, no, enko, 80, E]","((singleeq, 493), (commonsensqa, 1050), (coin_flip, 378), (last_letters, 226), (gsm8k, 1219), (aqua, 92))",6,"Q1: 37 classrooms are needed at the school.\nQ2: (E) trust himself\nQ3: Yes, the coin is still heads up.